In [ ]:
!pip install bilby
!pip install surmise

In [ ]:
#!/usr/bin/env python
"""
An example of how to use bilby to perform parameter estimation for
non-gravitational wave data. In this case, fitting a linear function to
data with background Gaussian noise

"""
import bilby
import matplotlib.pyplot as plt
import numpy as np

# A few simple setup steps
label = "linear_regression"
outdir = "outdir"
bilby.utils.check_directory_exists_and_if_not_mkdir(outdir)


# First, we define our "signal model", in this case a simple linear function
def model(time, m, c):
    return time * m + c


# Now we define the injection parameters which we make simulated data with
injection_parameters = dict(m=0.5, c=0.2)

# For this example, we'll use standard Gaussian noise

# These lines of code generate the fake data. Note the ** just unpacks the
# contents of the injection_parameters when calling the model function.
sampling_frequency = 10
time_duration = 10
time = np.arange(0, time_duration, 1 / sampling_frequency)
N = len(time)
sigma = np.random.normal(1, 0.01, N)
data = model(time, **injection_parameters) + np.random.normal(0, sigma, N)


# From hereon, the syntax is exactly equivalent to other bilby examples
# We make a prior
priors = dict()
priors["m"] = bilby.core.prior.Uniform(0, 5, "m")
priors["c"] = bilby.core.prior.Uniform(-2, 2, "c")


In [ ]:
class MultiUniform(bilby.core.prior.Prior):

    def __init__(self, minimum, maximum, name=None, latex_label=None):
        super(MultiUniform, self).__init__(
            name=name, latex_label=latex_label, minimum=minimum, maximum=maximum
        )

    def sample(self, size):
        U1 = bilby.core.prior.Uniform(self.minimum[0], self.maximum[0])
        U2 = bilby.core.prior.Uniform(self.minimum[1], self.maximum[1])
        theta = np.hstack((U1.sample(size).reshape(size, 1),
                           U2.sample(size).reshape(size, 1)))

        return theta

    def prob(self, val):
        in_prior = ((val >= self.minimum) & (val <= self.maximum)).all(axis=1)

        return in_prior

    @property
    def width(self):
        width = np.array(self.maximum) - np.array(self.minimum)
        return width

In [ ]:
priors_surmise = dict()
priors_surmise['theta'] = MultiUniform((-2, 0), (2, 5), 'theta')
priors_surmise['sigma'] = bilby.core.prior.Uniform(0.1, 1, 'sigma')

In [ ]:
theta_test = priors_surmise['theta'].sample(1)

print(theta_test.shape)
priors_surmise['theta'].prob(theta_test)


In [ ]:

likelihood = bilby.core.likelihood.GaussianLikelihood(time, data, model)



In [ ]:
likelihood.__dict__

In [ ]:
# SURMISE
ntheta = 1000

In [ ]:
theta = np.hstack((priors['m'].sample(ntheta).reshape(ntheta, 1), priors['c'].sample(ntheta).reshape(ntheta, 1)))

In [ ]:
f = np.zeros((len(time), ntheta))
for i in range(ntheta):
  f[:, i] = model(time, *theta[i])

print(f.shape)

In [ ]:
from surmise.emulation import emulator
emu = emulator(x=time, theta=theta, f=f)

In [ ]:
emupred = emu.predict(x=time, theta=np.array((0.5, 0.2)).reshape(1, -1))
emumean = emupred.mean()

In [ ]:
# We quickly plot the data to check it looks sensible
fig, ax = plt.subplots()
ax.plot(time, data, "o", label="data")
ax.plot(time, model(time, **injection_parameters), "--r", label="signal")
ax.plot(time, emumean, '-.b', label='prediction')
ax.set_xlabel("time")
ax.set_ylabel("y")
ax.legend()
fig.savefig("{}/{}_data.png".format(outdir, label))


In [ ]:
class GaussianLikelihoodSurmise(bilby.core.likelihood.Analytical1DLikelihood):
    def __init__(self, x, y, emu, theta=None, sigma=None, **kwargs):
        """
        A general Gaussian likelihood for known or unknown noise - the model
        parameters are inferred from the arguments of function

        Parameters
        ----------
        x, y: array_like
            The data to analyse
        function:
            The python function to fit to the data. Note, this must take the
            dependent variable as its first argument. The other arguments
            will require a prior and will be sampled over (unless a fixed
            value is given).
        sigma: None, float, array_like
            If None, the standard deviation of the noise is unknown and will be
            estimated (note: this requires a prior to be given for sigma). If
            not None, this defined the standard-deviation of the data points.
            This can either be a single float, or an array with length equal
            to that for `x` and `y`.
        """
        self.x = x
        self.y = y
        self.N = len(x)
        self.emu = emu
        self.sigma = sigma
        self.theta = theta

        super(GaussianLikelihoodSurmise, self).__init__(x=x, y=y, func=emu.predict, **kwargs)
        if self.sigma is None:
            self.parameters['sigma'] = None
        if self.theta is None:
            self.parameters['theta'] = None

        self._function_keys.remove('args')

    @property
    def residual(self):
        """ Residual of the function against the data. """
        return self.y - self.emu.predict(x=self.x, **self.model_parameters, **self.kwargs).mean().T


    def log_likelihood(self):
        sigma = self.sigma
        res = self.residual
        return -0.5 * (np.sum((res / sigma)**2)
                       + self.N*np.log(2*np.pi*sigma**2))

In [ ]:
priors_surmise['theta'].sample(100).shape

In [ ]:
likelihood_surmise = GaussianLikelihoodSurmise(x=time, y=data, emu=emu)

In [ ]:
likelihood_surmise.__dict__

In [ ]:
# And run sampler
result = bilby.run_sampler(
    likelihood=likelihood_surmise,
    priors=priors_surmise,
    sampler="dynesty",
    nlive=250,
    outdir=outdir,
    label=label,
)

# Finally plot a corner plot: all outputs are stored in outdir
result.plot_corner()
